In [1]:
import numpy as np
import neuprint
import pandas as pd
from neuprint import NeuronCriteria as NC

## Pulling Central Complex Data

In [2]:
token = "INSERT TOKEN HERE"

client = neuprint.Client('https://neuprint.janelia.org', token=token, dataset='hemibrain:v1.2.1')

order = ['PEN_a(PEN1)', 'PEN_b(PEN2)', 'PEG', 'EPG', 'Delta7']

neuron_criteria = NC(
    status='Traced', # Only traced neurons
    type=order,  
    cropped=False, # No cropped neurons
    inputRois=['EB','PB'], # Defines input regions of interest 
    outputRois=['EB','PB'], # Defines output regions of interest 
    roi_req='any' # Neurons that begin or end in either ROI are selected 

)
cx_neuron_df, cx_conn_df = neuprint.fetch_adjacencies(neuron_criteria, neuron_criteria, min_total_weight=5, rois=["EB","PB"])
cx_conn_df = neuprint.merge_neuron_properties(cx_neuron_df, cx_conn_df, ['type', 'instance'])


## Pulling Mushroom Body Data

In [3]:
w_thresh = 5

is_thresh = f"""AND w.weightHP >= {w_thresh}"""
types = ["PN", "KC", "APL", "MBON"]
wheres_thresh = []

table = pd.DataFrame()    
for c1 in types:
    for c2 in types:
        where = f"""(a.type CONTAINS "{c1}") AND (b.type CONTAINS "{c2}") {is_thresh}"""
        q = f" MATCH (a :`hemibrain_Neuron`)-[w:ConnectsTo]->(b:`hemibrain_Neuron`) WHERE {where} RETURN a.bodyId, a.type, b.bodyId, b.type, w.weight"

        lh_table = neuprint.fetch_custom(q)
        lh_table["Supertype_pre"] = c1
        lh_table["Supertype_post"] = c2
        table = pd.concat([table,lh_table])
table = table.rename(columns={"w.weight": "weight"})

## Ablation Methods

In [4]:
def preferential_detachment(weight_matrix, decay_vector, n_passes,p=.05, spread_rate=1):
    weights_mask = weight_matrix > 0
    
    sh = weight_matrix.shape
    for i in range(sh[0]):
        for j in range(sh[1]):
            if i == j:
                weights_mask[i,j] = 1
    osum = np.sum(weight_matrix)
    n_nodes = sh[0]
    avg1, avg2 = [], []
    decay_matrix = weights_mask 

    for i in range(n_passes):
        # Transmit decay
        decay_matrix = weights_mask * decay_vector 
        
        # binomial distribution of where to decrement weights

        decay_prob = p * (1/n_nodes) * weights_mask + (1 - p)*(decay_matrix)/osum

        decay_choose = np.random.random(sh) < decay_prob 
#         if i == 0:
#             print(np.mean(decay_choose), np.mean(decay_prob), 'decay')
        decay_vector = (decay_vector + np.sum(decay_choose,axis=1))* spread_rate
        # Maximum of one decrement per time step
        weight_matrix = weight_matrix - decay_choose
        weight_matrix[weight_matrix < 0] = 0
    return weight_matrix, decay_vector

def binomial_detachment(weight_matrix,decay_vector,n_pass,p=.1,spread_rate=1):
    # Degeneration model has base prob that an edge is deleted, p
    
    weights_mask = weight_matrix > 0    
    
    sh = weight_matrix.shape
    for i in range(sh[0]):
        for j in range(sh[1]):
            if i == j:
                weights_mask[i,j] = 1
    n_nodes = sh[0]
    osum = np.sum(weight_matrix)
    decay_matrix = weights_mask
    for i in range(n_pass):
        
        decay_matrix = weights_mask * decay_vector 
        if np.sum(decay_matrix) == 0:
            decay_chance = weights_mask * p / n_nodes
        else:
            decay_chance = weights_mask * p / n_nodes + (1-p) * decay_matrix / osum
#         print(weights_mask * p / n_nodes, (1-p) * spread_rate * decay_matrix / np.sum(decay_matrix))
#         if i == 0:
#             print(np.sum(decay_chance > p / n_nodes), decay_chance.shape[0]*decay_chance.shape[1], np.mean((1-p) * decay_matrix / osum))
        decay_chance[decay_chance>1]=1
        binomial_delete = np.random.binomial(weight_matrix, decay_chance)

        weight_matrix = weight_matrix - binomial_delete
#         print(decay_vector.shape, binomial_delete.shape)
        decay_vector = (decay_vector + np.sum(binomial_delete,axis=1))*spread_rate
        
    return weight_matrix, decay_vector

def random_detachment(weight_matrix,n_passes,p=.02):
    # Every synapse has an equal chance to be deleted, p
    sh = weight_matrix.shape
    weights_mask = weight_matrix > 0
#     print(sh, weight_matrix)
    for i in range(n_passes):
#         print(weight_matrix)
        delete = np.random.binomial(weight_matrix,p)
                
        weight_matrix = weight_matrix - delete
        
    return weight_matrix

## Defining Ablation loop and graph metrics obtained

In [5]:
import copy
from tqdm import tqdm
    
def test_network_degeneration(G, func, n_passes, weight_key, sum_min, b_or_c="B",decomp=False,dv=np.zeros(1),spread_rate=1):
    print("start", weight_key)
    G_decomp = G
    nodelist = G_decomp.nodes()
    decomp_df = pd.DataFrame()
    names = list(G_decomp.nodes)
    i=0
    m = nx.adjacency_matrix(G_decomp).toarray()
    sum_num = np.sum(m)
    edges = np.sum(m>0)
    n_edges=edges
    o_sum_num = sum_num
    while edges/n_edges > sum_min:

        if b_or_c=="B":
            G_b = nx.betweenness_centrality(G_decomp,weight=weight_key)
            data_df = pd.DataFrame(G_b,index=["Betweenness"]).T
        elif b_or_c=="s":
            G_b = nx.current_flow_betweenness_centrality(G_decomp.to_undirected(),weight=weight_key)
            data_df = pd.DataFrame(G_b,index=["Current"]).T
        elif b_or_c=="d":
            G_bi = nx.in_degree_centrality(G_decomp)
            G_bo = nx.out_degree_centrality(G_decomp)
            data_df = pd.DataFrame(G_bi,index=["In-Degree Cenrality"]).T
            data_df["Out-Degree Cenrality"] = G_bo
        else:
            G_b = nx.eigenvector_centrality(G_decomp,weight=weight_key)
            data_df = pd.DataFrame(G_b,index=["Eigen Cenrality"]).T
        data_df["Degree"] = data_df.index.map(G_decomp.degree)
        neighbor_deg = nx.average_neighbor_degree(G_decomp,weight=weight_key,source='in',target='out')
        data_df["Neighbor Degree"] = data_df.index.map(neighbor_deg)
#         print("D,ND")
#         data_df["Closeness Vitality"] = nx.closeness_vitality(cx_graph,weight='weight')
#         msts = np.product(np.linalg.eig(nx.laplacian_matrix(G_decomp.to_undirected()).toarray())[0])
#         data_df["Number of MSTs"] = msts
        data_df["Density"] = nx.density(G_decomp)
        data_df["Transitivity"] = nx.transitivity(G_decomp)
#         data_df["Modularity"] = nx.algorithms.community.modularity(G)
        data_df["Reciprocity"] = nx.reciprocity(G_decomp,nodes=nodelist)
#         print("D,T,R")
        data_df["Iter"] = i
        
        A = nx.adjacency_matrix(G_decomp,weight=weight_key).toarray()
#         print(G_decomp[5813039315], A, weight_key)
        sum_num=np.sum(A)
        edges=np.sum(A>0)
        data_df["Sum"] = sum_num
        data_df["Sum Fraction"] = sum_num/o_sum_num
        
        if not decomp:
            print(sum_num, np.sum(A > 0), sum_num/o_sum_num, np.max(A), np.min(A))

            A = func(A,n_passes)

        else:
            print(sum_num, np.sum(A > 0), sum_num/o_sum_num, np.max(A), np.min(A), np.sum(dv), np.sum(dv>0))

            A, dv = func(A,dv,n_passes,.05, spread_rate)
        ad = pd.DataFrame(A,columns = nodelist, index =nodelist)

        G_decomp=nx.from_pandas_adjacency(ad, create_using=nx.DiGraph())

        decomp_df = pd.concat([decomp_df, data_df])
        i+=1
    return decomp_df

## Preprocessing dataframes

In [6]:
import networkx as nx
cx_graph = nx.from_pandas_edgelist(cx_conn_df, source='bodyId_pre',target='bodyId_post',edge_attr=['weight','type_pre','type_post'],create_using=nx.DiGraph())
type_dict_cx = {}
for i in range(cx_neuron_df.shape[0]):
    row = cx_neuron_df.iloc[i]
    type_dict_cx[row.bodyId] = row.type
    
nx.set_node_attributes(cx_graph, type_dict_cx, "type")

mb_graph = nx.from_pandas_edgelist(table, source='a.bodyId',target='b.bodyId',edge_attr=['weight','a.type','b.type'],create_using=nx.DiGraph())
mb_ids = np.unique(list(table["a.bodyId"].unique()) + list(table["b.bodyId"].unique()))
mb_neurons = neuprint.fetch_neurons(NC(bodyId=mb_ids))

type_dict_mb = {}
for i in range(mb_neurons[0].shape[0]):
    row = mb_neurons[0].iloc[i]
    type_dict_mb[row.bodyId] = row.type
nx.set_node_attributes(mb_graph, type_dict_mb, "type")

In [23]:
df = pd.DataFrame(type_dict_mb, index=[0]).T
kc_ct = 0
pn_ct = 0
mbon_ct = 0
for i in range(df.shape[0]):
    if "PN" in df.iloc[i][0]:
        pn_ct += 1 
    if "KC" in df.iloc[i][0]:
        kc_ct += 1
    if "MBON" in df.iloc[i][0]:
        mbon_ct += 1
print(pn_ct, kc_ct, mbon_ct)

412 1927 68


## Creating Artificial Neural Network Structure

In [7]:
node_sizes = [64, 128, 256, 256, 512, 512, 512, 4096, 1000]
CNN_DG = nx.DiGraph()
node_count = 0
previous_node_list = []

for s, node_size in tqdm(enumerate(node_sizes)):
    current_node_list = []
    for i in range(node_size // 8):
        CNN_DG.add_node(node_count)
        current_node_list.append(node_count)
        node_count+=1
    if s > 0:
        for i, ival in enumerate(previous_node_list):
            for j, jval in enumerate(current_node_list):
                r = np.random.random(1)
#                 print(ival, len(previous_node_list), "|", jval, len(current_node_list))
                if r > .5:
                    CNN_DG.add_edge(ival, jval,weight=int(np.random.random()*100))
    else:
        first_node_list = current_node_list
    previous_node_list = current_node_list

for i, ival in enumerate(current_node_list):
    for j, jval in enumerate(first_node_list):
        r = np.random.random(1)
#                 print(ival, len(previous_node_list), "|", jval, len(current_node_list))
        if r > .5:
            CNN_DG.add_edge(ival, jval,weight=int(np.random.random()*100))  
#     print(len(previous_node_list), len(current_node_list))


9it [00:00, 21.74it/s]


In [8]:
mb_df_random = test_network_degeneration(G=mb_graph, func=random_detachment, n_passes=1, weight_key='weight', sum_min=.25, b_or_c="d")
cx_df_random = test_network_degeneration(G=cx_graph, func=random_detachment, n_passes=1, weight_key='weight', sum_min=.25, b_or_c="d")
CNN_df_random = test_network_degeneration(G=CNN_DG, func=random_detachment, n_passes=1, weight_key='weight', sum_min=.25, b_or_c="d")

start weight
824433 47872 1.0 613 0
808120 47872 0.9802130676477046 602 0
791829 47872 0.960452820301953 591 0
776234 47872 0.9415367895268627 572 0
760757 47872 0.9227638874232351 561 0
745614 47872 0.9043961122371376 553 0
730682 47872 0.8862842705228927 542 0
716257 47872 0.8687873969140003 532 0
701979 47872 0.8514688276670148 520 0
687985 47872 0.834494737595414 506 0
674075 47872 0.8176225357306173 497 0
660337 47872 0.8009589620988 484 0
647010 47871 0.7847939129074164 471 0
633961 47871 0.7689660651623601 459 0
621352 47868 0.7536719175481816 452 0
608975 47866 0.7386591754575569 440 0
596872 47862 0.723978782993888 431 0
584998 47861 0.709576157189244 424 0
573309 47859 0.6953979280305373 420 0
561855 47852 0.6815047432599132 411 0
550600 47847 0.6678529365030269 399 0
539837 47842 0.6547979035288495 391 0
528990 47837 0.6416409823478681 382 0
518453 47835 0.6288600771681871 374 0
508126 47826 0.616333892505516 371 0
498138 47812 0.6042188995345892 366 0
487970 47801 0.5918855

67232 5304 0.6524211547792333 121 0
65905 5302 0.63954391072295 118 0
64605 5300 0.6269286754002911 114 0
63311 5299 0.6143716642406599 112 0
62073 5297 0.6023580786026201 110 0
60836 5294 0.5903541969917516 109 0
59620 5292 0.5785540999514799 108 0
58457 5291 0.5672683163512858 108 0
57258 5287 0.5556331877729258 106 0
56104 5284 0.5444347404172731 103 0
54985 5280 0.5335759340126153 97 0
53917 5275 0.5232120329936923 93 0
52827 5271 0.5126346433770015 89 0
51731 5269 0.5019990295972828 88 0
50701 5266 0.4920038816108685 86 0
49658 5262 0.4818825812712276 84 0
48690 5258 0.4724890829694323 82 0
47691 5256 0.46279475982532753 80 0
46744 5249 0.45360504609412905 80 0
45803 5247 0.44447355652595827 79 0
44937 5244 0.4360698689956332 79 0
43992 5242 0.4268995633187773 78 0
43089 5236 0.418136826783115 76 0
42250 5233 0.40999514798641434 75 0
41404 5225 0.4017855409995148 71 0
40559 5218 0.3935856380397865 71 0
39767 5213 0.3859000485201359 70 0
39003 5210 0.3784861717612809 68 0
38257 520

1033571 53367 0.37934974341055855 54 0
1012758 53337 0.37171078468435204 53 0
992492 53307 0.36427259040456056 53 0
972714 53272 0.3570135059051173 52 0
953416 53246 0.349930594960115 52 0
934297 53221 0.34291338206978966 50 0
915823 53185 0.33613290239324434 50 0
897820 53162 0.3295252930169941 50 0
879767 53126 0.322899332228823 50 0
861981 53101 0.3163713679803097 49 0
844881 53066 0.3100951851033515 49 0
827765 53025 0.3038131297745786 49 0
811447 52993 0.29782396298006375 48 0
795323 52963 0.29190599966380215 48 0
779400 52923 0.2860618090234626 48 0
763691 52881 0.28029616242614475 47 0
748448 52846 0.2747015509879299 47 0
733445 52803 0.26919502632693554 46 0
718909 52768 0.2638599038532827 46 0
704442 52729 0.2585501063280807 46 0
690479 52681 0.25342529103504163 45 0
676772 52633 0.2483944349710378 44 0
663155 52575 0.24339661144849162 43 0
649793 52538 0.2384923801267422 39 0
636844 52487 0.23373973146745963 39 0
624128 52449 0.22907260038772864 39 0
611672 52409 0.2245008966

In [9]:
cx_df_random.to_csv("cx_random1.csv")
mb_df_random.to_csv("mb_random1.csv")
CNN_df_random.to_csv("CNN_random1.csv")

In [10]:
cx_detach, mb_detach, CNN_detach = np.zeros(len(cx_graph)), np.zeros(len(mb_graph)), np.zeros(len(CNN_DG))
cx_df_CX_binomial_degen = test_network_degeneration(G=cx_graph, func=binomial_detachment, n_passes=10, weight_key='weight', sum_min=.25, b_or_c="d", decomp=True, dv=cx_detach, spread_rate=1.01)
mb_df_CX_binomial_degen = test_network_degeneration(G=mb_graph, func=binomial_detachment, n_passes=10, weight_key='weight', sum_min=.25, b_or_c="d", decomp=True, dv=mb_detach, spread_rate=1.01)
CNN_df_CX_binomial_degen = test_network_degeneration(G=CNN_DG, func=binomial_detachment, n_passes=10, weight_key='weight', sum_min=.25, b_or_c="d", decomp=True, dv=CNN_detach,spread_rate=1.01)

start weight
103050 5326 1.0 184 0 0.0 0
102696 5324 0.9965647743813683 184 0 374.1137495132335 123
102291 5323 0.9926346433770015 181 0 841.1429905687482 136
101876 5321 0.9886074721009219 181 0 1366.8364105131 137
101370 5321 0.9836972343522562 180 0 2043.5404198490096 141
100884 5321 0.978981077147016 180 0 2770.8696314325357 143
100317 5321 0.9734788937409025 179 0 3658.452838768296 145
99735 5321 0.9678311499272197 179 0 4654.404982647295 145
99008 5321 0.9607763221737021 178 0 5909.702333238165 148
98248 5320 0.9534012615235322 176 0 7330.772475989566 148
97329 5320 0.9444832605531296 176 0 9066.200151382927 148
96291 5320 0.9344104803493449 174 0 11111.97386282146 148
95083 5320 0.9226880155264435 172 0 13548.95767093193 148
93723 5319 0.909490538573508 168 0 16400.642537531236 148
92133 5316 0.894061135371179 162 0 19793.168718624685 148
90304 5316 0.876312469674915 160 0 23794.073966062882 148
88161 5314 0.8555167394468705 152 0 28547.348810252195 148
85749 5311 0.832110625909

2691316 54240 0.9877889705078129 99 0 101477.18158857083 895
2689263 54240 0.9870354615343395 99 0 114262.3331433362 897
2687099 54240 0.9862412124264016 99 0 128504.13855876819 898
2684884 54239 0.9854282448783045 99 0 144288.26673227004 901
2682557 54238 0.9845741701675044 99 0 161840.03480361623 903
2680102 54237 0.9836731158421866 99 0 181366.1004680349 903
2677570 54237 0.9827438003425107 99 0 203014.7219963749 905
2674907 54237 0.981766404143602 99 0 227069.83058177997 906
2672138 54235 0.9807501029514208 99 0 253751.6254576128 906
2669194 54234 0.9796695718175165 99 0 283408.1530032917 906
2666079 54233 0.9785262788548426 99 0 316350.8325490374 906
2662778 54232 0.9773147186398227 99 0 352936.0010913852 907
2659273 54232 0.9760282846641655 99 0 393563.62219951267 907
2655479 54232 0.9746357795276053 99 0 438750.3029591953 907
2651582 54230 0.973205470482488 99 0 488768.69839697925 907
2647315 54228 0.9716393609891558 99 0 544411.5437052648 907
2642801 54228 0.9699825955209342 99

In [11]:
cx_df_CX_binomial_degen.to_csv("cx_random_binomial_degen.csv")
mb_df_CX_binomial_degen.to_csv("mb_random_binomial_degen.csv")
CNN_df_CX_binomial_degen.to_csv("CNN_random_binomial_degen.csv")

In [12]:
cx_detach, mb_detach, CNN_detach = np.zeros(len(cx_graph)), np.zeros(len(mb_graph)), np.zeros(len(CNN_DG))
cx_df_CX_preferential_degen = test_network_degeneration(G=cx_graph, func=preferential_detachment, n_passes=10, weight_key='weight', sum_min=.25, b_or_c="d", decomp=True, dv=cx_detach,spread_rate=1.01)
mb_df_CX_preferential_degen = test_network_degeneration(G=mb_graph, func=preferential_detachment, n_passes=10, weight_key='weight', sum_min=.25, b_or_c="d", decomp=True, dv=mb_detach,spread_rate=1.01)
CNN_df_CX_preferential_degen = test_network_degeneration(G=CNN_DG, func=preferential_detachment, n_passes=10, weight_key='weight', sum_min=.25, b_or_c="d", decomp=True, dv=CNN_detach,spread_rate=1.01)

start weight
103050 5326 1.0 184 0 0.0 0
103032 5326 0.999825327510917 184 0 19.176512469101898 17
103011 5326 0.9996215429403202 184 0 43.311875200992446 33
102990 5325 0.9994177583697235 184 0 69.8589043225168 49
102977 5325 0.9992916060164968 184 0 92.05639711924132 59
102965 5325 0.9991751576904415 184 0 115.39013172832395 65
102949 5325 0.9990198932557011 184 0 145.26399799347985 74
102936 5325 0.9988937409024745 184 0 174.1406809760183 79
102917 5325 0.9987093643862203 184 0 212.41938366643143 86
102897 5325 0.9985152838427948 184 0 256.6134174896178 92
102877 5325 0.9983212032993692 184 0 305.68842252844775 99
102857 5324 0.9981271227559437 184 0 359.82343358892285 108
102838 5324 0.9979427462396895 184 0 417.5204763691358 110
102815 5324 0.9977195536147502 184 0 485.43710863215125 112
102803 5324 0.9976031052886948 184 0 549.9031173872485 114
102791 5324 0.9974866569626395 184 0 620.1887278806432 114
102775 5324 0.997331392527899 184 0 704.1669273715792 116
102749 5324 0.997079

823885 47872 0.9993353007460886 613 0 10499.98967652961 465
823867 47872 0.9993134675589163 613 0 11617.45781660929 478
823850 47872 0.9992928473265869 613 0 12852.991821943517 491
823822 47872 0.9992588845909856 613 0 14227.433054252584 510
823802 47872 0.9992346254941274 613 0 15736.914372101786 523
823768 47872 0.9991933850294688 613 0 17419.367536586797 550
823722 47872 0.9991375891066951 613 0 19292.45583568143 585
823689 47872 0.9990975615968793 613 0 21346.75043944488 615
823648 47872 0.9990478304483202 613 0 23625.673416238427 641
823614 47872 0.9990065899836615 613 0 26136.29974264892 664
823574 47872 0.9989580717899453 613 0 28913.13794924677 692
823536 47872 0.9989119795059149 613 0 31978.194037092668 715
823491 47872 0.9988573965379843 613 0 35371.186738712815 750
823430 47872 0.9987834062925671 613 0 39136.24584512985 785
823379 47872 0.998721545595579 613 0 43285.87239199702 815
823324 47872 0.9986548330792193 613 0 47873.679725537106 843
823262 47872 0.9985796298789592 6

2724471 54249 0.9999577917525818 99 0 144.57971131586442 96
2724435 54248 0.9999445787359987 99 0 197.69400027057574 119
2724388 54247 0.9999273284087932 99 0 269.0940373541819 155
2724353 54247 0.9999144824204484 99 0 334.08764208407 177
2724317 54247 0.9999012694038654 99 0 406.7995337429704 198
2724279 54247 0.9998873223308055 99 0 489.5468557372719 221
2724246 54247 0.9998752103989377 99 0 575.8308841707654 242
2724215 54247 0.9998638325235467 99 0 669.0260751497708 258
2724181 54245 0.9998513535634405 99 0 774.6758545486281 272
2724144 54244 0.999837773518619 99 0 895.9707842981911 290
2724125 54244 0.999830799982089 99 0 1009.6764760232489 298
2724088 54244 0.9998172199372676 99 0 1155.1605972523305 316
2724074 54244 0.9998120815419297 99 0 1291.8387839145416 324
2724042 54243 0.9998003366383003 99 0 1461.778774443981 335
2724011 54243 0.9997889587629093 99 0 1647.3383208956684 350
2723977 54242 0.999776479802803 99 0 1859.0417424898724 367
2723952 54242 0.9997673040968426 99 0 2

2336068 49154 0.8574029228660794 99 0 411310325.71615255 879
2307846 48675 0.8470446519214295 99 0 454375050.2714364 879
2278521 48168 0.8362815488298039 99 0 501946829.77790594 879
2248480 47621 0.8252556535194705 99 0 554496702.137616 882
2217804 47042 0.8139966952777413 99 0 612545426.0093501 884
2186993 46413 0.8026881882238256 99 0 676667878.2905453 885
2156478 45740 0.7914883215284818 99 0 747499089.9981339 888
2126568 45085 0.7805105069173812 99 0 825740206.897155 890
2097365 44425 0.7697921812708426 99 0 912166620.7832348 891
2068649 43766 0.7592525983764139 99 0 1007634909.3313653 891
2041185 43137 0.7491725348364853 99 0 1113090186.595121 891
2014831 42497 0.7394998726412013 99 0 1229577327.40375 892
1989063 41939 0.7300422889936306 99 0 1358250890.4621277 892
1963728 41386 0.7207436285732952 99 0 1500386238.3346338 893
1939188 40839 0.7117367556025025 99 0 1657391492.7723765 895
1915280 40304 0.7029618444783905 99 0 1830822465.5566497 896
1891978 39852 0.69440935246676 99 0 

In [13]:
cx_df_CX_preferential_degen.to_csv("cx_df_preferential_degen.csv")
mb_df_CX_preferential_degen.to_csv("mb_df_preferential_degen.csv")
CNN_df_CX_preferential_degen.to_csv("CNN_df_preferential_degen.csv")